In [ ]:
%run my_import

uri = socket.gethostname()+".local:5555"
#uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"
#uri = "localhost:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))

In [ ]:
bId = 1

homing = {"type": "TRJ_CMD",
        "trajectory_cmd": {
                "type": "HOMING", "name": "homing", "board_id": bId, "homing_par": {"x": [0,3]}}
}

xs = [0,1,2,3,4]
xs = [x*2 for x in xs] 
smooth = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": bId,
                             "smooth_par": {"x": xs, "y": [0, 0.2, 0, -0.2, 0]}}} 
sine = {"type": "TRJ_CMD",
        "trajectory_cmd": {"type": "SINE", "name": "sine", "board_id": bId,
                           "period_par": {"freq": 0.05, "ampl": 2.0, "teta": 0, "secs": 500}}}
sine_ = {"type": "TRJ_CMD",
        "trajectory_cmd": {"type": "SINE", "name": "sine", "board_id": bId,
                           "period_par": {"freq": 0.2, "ampl": 1.0, "teta": 0, "secs": 500}}}

add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}
add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth"]}}
add_sine =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["sine"]}}
add_all =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing","smooth","sine"]}}

In [ ]:
iD = 1
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))

In [ ]:
sf =sdo_filter(['m3_fw_ver','c28_fw_ver','hw_config_low','hw_config_high'],sdo_dict)
print(sf)
hex( (sf['hw_config_high']<<16) + sf['hw_config_low'] )

In [ ]:
sdo = ['torque','motor_pos','link_pos','Min_pos','Max_pos','Max_cur']
#sdo = ['link_pos','fault','linkEncBadReadPPM','absLinkEncErr']
reply_cmd(SdoCmd(rd_sdo=sdo,wr_sdo={}).set_bid(iD))


In [ ]:
reply = io.doit(ctrl_cmd_set_home.set_bid(iD))

In [ ]:
#io.doit(ctrl_cmd_set_zero.set_value(3.14159).set_bid(bId))

In [ ]:
#reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0x3B).set_gains(u'POSITION',200,0,15,0,0,0))
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0x3B))

In [ ]:
ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
# define trajectories
reply = io.doit(homing)
reply = io.doit(smooth)
reply = io.doit(sine)

In [ ]:
#reply = io.doit(add_homing)
#reply = io.doit(add_smooth)
#reply = io.doit(add_sine)
reply = io.doit(add_all)

In [ ]:
ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:

reply = io.doit(ctrl_cmd_stop.set_bid(bId))

In [ ]:
ctrl_status_cmd_str('LED_ON', iD)

In [ ]:
reply = io.doit(ctrl_cmd_start().set_bid(bId).set_value(0xD4))

In [ ]:
io.debug=1
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 1
gains['kd'] = 0
gains['ki'] = 0
gains['kkp'] = 5
gains['kkd'] = 0.01
gains['kki'] = 0.8
reply = io.doit(ctrl_cmd_start().set_bid(bId).set_value(0xD4).set_gains(*gains.values()))

In [ ]:
cmd

In [ ]:
xs

In [ ]:
gains.values()

In [ ]:
io.doit(ctrl_cmd_fan.set_bid(bId).set_value(1))

In [ ]:
reply